# Validation

In order to validate our results, this notebook will show the flood mapping for the same area and time range from the ['Global Flood Monitoring'(GFM)](https://services.eodc.eu/browser/#/v1/collections/GFM) to allow comparisons.

In [ ]:
import hvplot.xarray
import pystac_client
from odc import stac as odc_stac
from dask.distributed import Client, wait

client = Client(processes=False, threads_per_worker=2, n_workers=3, memory_limit="12GB")

# Connect to STAC catalog
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1/")

# Search for available images
time_range = "2022-10-11/2022-10-25"
bounding_box = [12.3, 54.3, 13.1, 54.6]
search = eodc_catalog.search(collections="GFM", bbox=bounding_box, datetime=time_range)
items_GFM = search.item_collection()

print(f"Found {len(items_GFM)} items") 

### Load the data

In [ ]:
import pyproj
crs = pyproj.CRS.from_wkt(items_GFM[0].properties["proj:wkt2"])

# Set the resolution of the data
resolution = items_GFM[0].properties['gsd']

GFM_dc= odc_stac.load(
    items_GFM, 
    bbox=bounding_box,   # Define the bounding box for the area of interest
    crs=crs,   # Set the coordinate reference system
    bands=["tuw_likelihood","tuw_flood_extent"],   # Specify the bands to load
    resolution=resolution,   # Set the resolution of the data
    dtype='uint8',   # Define the data type
    chunks={"x": 1000, "y": 1000, "time": -1},  # Set the chunk size for Dask
)



### Pre-processing 

In [ ]:
GFM_dc["tuw_flood_extent"] = GFM_dc.tuw_flood_extent.where(GFM_dc.tuw_flood_extent!=255).compute()

#GFM_dc["tuw_flood_extent"] 

In [ ]:
GFM_dc_flood = GFM_dc["tuw_flood_extent"]

GFM_dc_flood.data

In [ ]:
import rioxarray  # This activates the .rio accessor for your DataArray

band_name = "tuw_flood_extent"
print(GFM_dc[band_name])
print("crs #######")
print(GFM_dc[band_name].rio.crs)  # Check if a CRS is assigned
print("coords #######")
print(GFM_dc[band_name].coords)   # Show available coordinates


In [ ]:
import numpy as np

values = GFM_dc[band_name].values
unique_values, counts = np.unique(values[~np.isnan(values)], return_counts=True)
print(dict(zip(unique_values, counts)))


In [ ]:
import rioxarray
tuw_flood = GFM_dc[band_name]

fd_reprojected = tuw_flood.rio.reproject("EPSG:4326")

values = GFM_dc[band_name].values
unique_values, counts = np.unique(values[~np.isnan(values)], return_counts=True)
print(dict(zip(unique_values, counts)))

fd_reprojected.hvplot.image(x="x", y="y", title=band_name, cmap=["rgba(0, 0, 1, 0.1)", "darkred"]).opts(frame_height=400, frame_width= 700)

### Plot



In [ ]:
from bokeh.models import FixedTicker
import hvplot.xarray

band_name = "tuw_flood_extent"
print(f"Visualizing band: {band_name}")

colorbar_opts = {
    "major_label_overrides": {
        0: "non-flood",
        1: "flood",
    },
    "ticker": FixedTicker(ticks=[0, 1]),
}
#tuw_flood = GFM_dc[band_name]#.where(wcover != 80)
tuw_flood_plot =tuw_flood.hvplot.image(x="x", y="y", title=band_name, cmap=["rgba(0, 0, 1, 0.1)", "darkred"]).opts(frame_height=400, frame_width= 700)
tuw_flood_plot



## 2nd plot

In [ ]:
from dask_flood_mapper import flood
from dask.distributed import Client

# define dask client
client = Client(processes=False, threads_per_worker=2, n_workers=1, memory_limit="12GB")

# flood area and time
time_range = "2022-10-11/2022-10-25"
bbox = [12.3, 54.3, 13.1, 54.6]
#calculate flood decision
fd = flood.decision(bbox=bbox, datetime=time_range).compute()

from importlib.resources import files
import xarray as xr

data_text = files("dask_flood_mapper.data").joinpath("wcover.tif")
wcover = xr.open_dataarray(data_text)

from bokeh.models import FixedTicker
import hvplot.xarray



In [ ]:
fd = fd.where(wcover != 80)
colorbar_opts = {
    "major_label_overrides": {
        0: "non-flood",
        1: "flood",
    },
    "ticker": FixedTicker(ticks=[0, 1]),
}
plot_obj=fd.hvplot.quadmesh(
    x="x",
    y="y",
    rasterize=True,
    geo=True,
    cmap=["rgba(0, 0, 1, 0.1)", "darkred"],
).opts(frame_height=400, frame_width= 700)#, colorbar_opts={**colorbar_opts})

plot_obj

In [ ]:
combined_plot = tuw_flood_plot + plot_obj
#combined_plot.opts(
 #   title="Comparison of Flood Extent and Water Cover",height=800,width=400)
combined_plot